# Audio-Dateia Transkribiera

Mit dem Code kå mr a Audio-Datei transkribiera:

- Schritt I): Modell wähla. Je graeßer s Modell, omso gnaoer d Transkriptio. Allerdings kå s sae, dass bei graeßere Modelle de Rechakapazitäta nemme ausroechat. *Konkret:* an Rechner mit 8GB Arbaetsspeicher vrpackt grad no s Modell `medium` (zor Sicherhaet älle andere Programme åschiaßa), an Rechner mit 16GB Arbaetsspeicher vrpackt nå au s Modell `large`.
- Schritt II): Datei auswähla. Standardmäßig werdat MP3-Dateia vorgschlaga. Mr kå bei dr Dateiauswahl rechts-onta ao andere Dateiendonga ågea.

Dia Word-Datei wird em gleicha Ordner onter am gleicha Dateinåma wia d ursprünglich Audio-Datei gspeicherat.

D Dateigröß därf **höchstens 25MB** betraga.

<div class="alert alert-block alert-info"> <b>Hinweis:</b> Des Notebook transkribiert bloß (Oxford-)Deutsch. </div>

Rest oefach laofa lao:

In [1]:
from pathlib import Path
import FreeSimpleGUI as sg
from docx import Document
import whisper
from pytictoc import TicToc
import warnings
import sys
# user input
path_start = Path(r'.')
my_language = 'de'
my_model = 'large'
warnings.simplefilter(action='ignore', category=Warning)

# Auswahl Modell, Window 1
layout = [[sg.T(""), sg.Radio('tiny (39 M)', "RADIO1", default=False, key="tiny")],
          [sg.T(""), sg.Radio('base (74 M)', "RADIO1", default=False, key="base")],
          [sg.T(""), sg.Radio('small (244 M)', "RADIO1", default=False, key="small")],
          [sg.T(""), sg.Radio('medium (769 M)', "RADIO1", default=False, key="medium")],
          [sg.T(""), sg.Radio('large (1550 M)', "RADIO1", default=True, key="large")],
          [sg.T(""), sg.Radio('small.en (244 M)', "RADIO1", default=True, key="small.en")],
          [sg.T(""), sg.Radio('medium.en (769 M)', "RADIO1", default=True, key="medium.en")],
          [sg.Button('OK'), sg.Button('Abbruch')]
          ]
window = sg.Window('Auswahl Modell', layout, size=(300, 200))
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == "Abbruch":
        print('Abbruch durch Benutzer, Vorgang wiederholen.')
        break
    elif event == 'OK':
        break
window.close()

# Modell laden, Datei auswählen
if event == 'OK':
    # Modell aus Eingabemaske abfangen
    for v in values:
        if values[v]:
            my_model = v
    print('Lade Modell:', my_model)
    try:
        whisper_model = whisper.load_model(my_model)
    except:
        print('Unbekannter Fehler.')

    if 'whisper_model' in globals():
        print('erfolgreich')
        # Datei auswählen
        layout = [[sg.Input(key='_FILE_'),
                   sg.FileBrowse(file_types=(("MP3", ".mp3"), ("*", ".*")), button_text="Auswahl", initial_folder=path_start)],
                  [sg.Button('OK'), sg.Button('Abbruch')]]
        window = sg.Window('Auswahl Audio-Datei', layout)
        event2, values2 = window.read()
        window.close()
    
        if event2 == 'OK':
            t = TicToc()
            t.tic()
            print('Starte Transkription')
            try:
                result = whisper_model.transcribe(values2['_FILE_'], language=my_language, fp16=False)
                print('erfolgreich')
                t.toc()
                # save file
                print('Word-Datei speichern')
                document = Document()
                document.add_paragraph(result["text"])
                try:
                    document.save(str(Path(values2['_FILE_']).parent / Path(values2['_FILE_']).stem) + '.docx')
                    print('erfolgreich')
                    print('Transkription beendet.')
                except PermissionError:
                    print(f'Datei speichern fehlgeschlagen. Möglicherweise ist die Datei  {Path(values2["_FILE_"]).stem}.docx  noch geöffnet.')
    
            except RuntimeError:
                print('Datei einlesen fehlgeschlagen, Dateiformat überprüfen.')
    
        else:
            print('Keine Datei ausgewählt, Vorgang wiederholen.')

Lade Modell: tiny
erfolgreich
Starte Transkription
erfolgreich
Elapsed time is 11.417688 seconds.
Word-Datei speichern
erfolgreich
Transkription beendet.


## Abschlussbemerkonga:

- Für a ordentliche Transkriptio braucht mr für Deutsch s Modell `medium`, s `large` ischd aegentlich no besser.
- Für Englisch kå mr a Nommer kloener ganga ond mit `medium.en` kriagt mr scho super Ergebnisse.
- Mir persönlich håt bei maera Zeit bei Fraunhofer scho gholfa, mô mir (fiktive) Anamnese-Gespräche vo Hausärzte ausgwertat hent. Der håt nå sogar de Krankata ond Medikamente (fast emmer) perfekt ausglesa.
- De Modelle, send aegentlich net für a CPU ausglegt, sondern für a GPU. Auf ara **GPU** spart mr für d Transkriptio etwa 80-90% vo dr Rechazeit. Allerdings sprengt des dr Råhma, wia mr so ebbas en Python aerichtat, ond s håt ao net jeder a GPU mit 16GB en saera Kist aebaut.

Wer a weng programmiera kå, der därf jetzt no ens nächste Notebook sprenga.